In [4]:
import pandas as pd 
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/7_1_1.csv')
df['신고'] = pd.to_datetime(df['신고일자'].astype(str).str.zfill(8) + df['신고시각'].astype(str).str.zfill(6), format='%Y%m%d%H%M%S')
df['처리'] = pd.to_datetime(df['처리일자'].astype(str).str.zfill(8) + df['처리시각'].astype(str).str.zfill(6), format='%Y%m%d%H%M%S')
df['소요'] = (df.처리 - df.신고).dt.total_seconds()
print(df.groupby('공장명')['소요'].mean().sort_values().reset_index().iloc[2]['공장명'])

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/7_1_2.csv')
df['구'] = df['STATION_ADDR1'].str.split(' ').str[1]
rs = round(df.groupby('구')['dist'].mean(),2).reset_index()
rs = rs[(rs['구'] == '마포구') | (rs['구'] == '성동구')].reset_index(drop=True)
print(rs.head(2))

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/7_1_3.csv')
print(df.head(2))
df['판매량'] = df['제품A'] + df['제품B'] + df['제품C'] + df['제품D'] + df['제품E']
df['연도'] = df['기간'].str[:4]
df['월'] = df['기간'].str[6:8].str.replace('월', '').astype(int)
df['분기'] =  np.ceil(df['월'] / 3) 
print(df.groupby(['연도', '분기'])['판매량'].mean().idxmax())

공장D
     구     dist
0  마포구  3045.88
1  성동구  3202.31
         기간  제품A  제품B  제품C  제품D  제품E
0  2018년_1월  890  567  274  762  691
1  2018년_2월  270  831  930  539  393
('2018', 3.0)


In [82]:
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/7_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/7_2_test.csv')

key = test['ID']
y_train = train['Target']
y_test = test['Target']
x_train = train.drop(['ID', 'level_0', 'Target'], axis=1)
x_test = test.drop(['ID', 'level_0', 'Target'], axis=1)

from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
concat = pd.concat([x_train, x_test], axis=0)


bo = ['Daytime/evening attendance', 'Educational special needs', 'Gender']
#obj = ['Marital Status', 'Application mode', 'Course', 'Previous qualification', 'Nacionality', "Mother's qualification", "Father's qualification", "Mother's occupation", "Father's occupation"]
obj = ['Marital Status', 'Application mode', 'Course', 'Previous qualification', 'Nacionality', "Mother's qualification", "Father's qualification", "Mother's occupation", "Father's occupation"]
num = ['Curricular units 1st sem (credited)', 'Curricular units 1st sem (enrolled)', 'Curricular units 1st sem (evaluations)', 'Curricular units 1st sem (approved)']

for col in bo:
    concat[col] = concat[col].map({'1_':True, '0_':False})
concat = pd.get_dummies(concat, columns = obj)
scaler = StandardScaler()
#scaler = MinMaxScaler()
concat[num] = scaler.fit_transform(concat[num])

x_train = concat.iloc[:len(x_train),:]
x_test = concat.iloc[len(x_train):,:]

x_t, x_va, y_t, y_va = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)

model = RandomForestClassifier(
    n_estimators = 1000,
    max_depth = None,
    min_samples_leaf = 5,
    min_samples_split = 6,
    random_state= 42,
    class_weight = 'balanced'
)

model.fit(x_t, y_t)
pred = model.predict(x_va)
f1 = f1_score(y_va, pred, average='macro')
print('F1 Score:', f1)

result = pd.DataFrame()
result['ID'] = key
result['pred'] = model.predict(x_test)

print(result.head(3))
print(result.isnull().sum())
#result.to_csv('result.csv', index=False)

F1 Score: 0.5875816993464053
     ID      pred
0  1903  Graduate
1   796  Graduate
2  1060  Graduate
ID      0
pred    0
dtype: int64


In [94]:
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/7_3_1.csv')
print(df.head(2))
print(round(df.before.mean(),2), round(df.before.std(),2), round(df.after.mean(),2), round(df.after.std(),2))
from scipy.stats import ttest_rel
stats, p = ttest_rel(df.before, df.after)
print(round(stats,2))
if p < 0.05:
    print('기각')
    print(round(p, 3))
else:
    print('채택')
    print(round(p, 3))

train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/7_3_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/7_3_2_test.csv')

print(train.head(5))

import statsmodels.formula.api as smf

model = smf.logit('purchase ~ age + income + family_members', data = train).fit()
import numpy as np
print(np.exp(model.params['income']))
print(-2 * model.llf)

pred = model.predict(test)
pred = (pred > 0.5).astype(int)
print(np.mean(pred != test['purchase']))

      before      after
0  87.640523  88.163191
1  74.001572  80.936085
71.41 11.37 76.3 11.94
-7.9
기각
0.0
   age        income  family_members  purchase
0   37  43620.800179               2         0
1   56  60865.620200               3         1
2   21  39500.939224               2         0
3   19  48343.513116               2         0
4   57  36974.711408               1         0
Optimization terminated successfully.
         Current function value: 0.676174
         Iterations 4
1.0000152084810885
108.18783555703052
0.35
